In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

In [2]:
# 通过下面命令可以知道显卡型号
!nvidia-smi

In [3]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

print(x_valid.shape, y_valid.shape)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(
    x_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28, 1)
x_valid_scaled = scaler.transform(
    x_valid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28, 1)
x_test_scaled = scaler.transform(
    x_test.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28, 1)

In [5]:
a=np.arange(1,10).reshape(3,3)
print(a)
b=a.reshape(3,3,1)
print(b)
print(b[1])

In [6]:
x_train_scaled[1]

In [7]:
model = keras.models.Sequential()
# 添加卷积层，filters输出有多少通道，就是有多少卷积核，kernel_size卷积核的大小，
# padding = same代表输出和输入大小一样，
# 1代表通道数目是1
model.add(keras.layers.Conv2D(filters=32,  # 通道数，有多少通道数就有多少个卷积核
                              kernel_size=3,  # 卷积核尺寸，3*3，其实是3*3*1
                              padding='same',
                              activation='relu',
                              input_shape=(28, 28, 1)))
model.add(keras.layers.Conv2D(filters=32, 
                              kernel_size=3,  # 上一层输出的是28*28*32，这里卷积核尺寸为3*3*32
                              padding='same',
                              activation='relu'))
# 添加池化层，pool_size是窗口大小，步长默认和窗口大小相等
model.add(keras.layers.MaxPool2D(pool_size=2))
# 为了缓解前面池化带来的损失，所以filters翻倍，从更多角度看这张图
model.add(keras.layers.Conv2D(filters=64, kernel_size=3,
                              padding='same',
                              activation='relu'))
model.add(keras.layers.Conv2D(filters=64, kernel_size=3,
                              padding='same',
                              activation='relu'))
model.add(keras.layers.MaxPool2D(pool_size=(2,2)))
model.add(keras.layers.Conv2D(filters=128, kernel_size=3,
                              padding='same',
                              activation='relu'))
model.add(keras.layers.Conv2D(filters=128, kernel_size=3,
                              padding='same',
                              activation='relu'))
model.add(keras.layers.MaxPool2D(pool_size=2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))  # 选了个中间合适值过度，不然直接降到10有些太快了
model.add(keras.layers.Dense(10, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy",
              optimizer = "sgd",
              metrics = ["accuracy"])

In [8]:
3*3*32*32+32  # 3*3*32卷积核（类似最开始的3*3*1），这个32是前一层输出的通道数；32是32个通道数,有多少通道数就有多少个卷积核；32是每一个卷积核都有一个偏置

In [9]:
3*3*64*64+64

In [10]:
3*3*64*128+128

In [11]:
1+(28-2)//2

In [12]:
model.summary()

In [13]:
model.variables

In [14]:
!ls

In [15]:
!rm -rf cnn-relu-callbacks

In [16]:
!ls

In [17]:
logdir = './cnn-relu-callbacks'
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir,
                                 "fashion_mnist_model.h5")

callbacks = [
    keras.callbacks.TensorBoard(logdir),
    keras.callbacks.ModelCheckpoint(output_model_file,
                                    save_best_only = True),
    keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3),
]
history = model.fit(x_train_scaled, y_train, epochs=10,
                    validation_data=(x_valid_scaled, y_valid),
                    callbacks = callbacks)

In [18]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 1)
    plt.show()

plot_learning_curves(history)

In [19]:
model.evaluate(x_test_scaled, y_test, verbose = 0)

In [20]:
# 通过下面方法来确定某一层输出尺寸
x = tf.constant([[1., 2., 3.],
                 [4., 5., 6.],
                 [7., 8., 9.]])
x = tf.reshape(x, [1, 3, 3, 1])
max_pool_2d = tf.keras.layers.MaxPooling2D(pool_size=(2, 2),
   strides=(1, 1), padding='valid')

In [21]:
x

In [22]:
max_pool_2d(x)

In [23]:
gmp = tf.keras.layers.GlobalMaxPooling2D()
gmp(x)

In [26]:
!ls

In [25]:
!tar cvzf cnn-relu-callbacks.tar.gz cnn-relu-callbacks